In [96]:
import numpy as np

In [97]:
# DIPOLE in units of Debye, which is 0.393430307 au. The atomic unit of the dipole is 1 au of dipole = ea_0.
# EFIELD in units of au, 1 au of efield is e/(a_0^2).
# TIME in units of au, 1 au of time is a_0/(α·c).
#     FREQUENCY in units of cm^-1, which is actually a wavenumber. To get in au units,
# multiply by 2π*100*c*2.418884326509e-17
field_data = np.load('E_FIELD_DAT.npy').astype(complex)
init_field_data = np.load('E_FIELD_INIT_DAT.npy').astype(complex)
moment_data = np.load('MOM_DAT.npy').astype(complex)*0.393430307
init_moment_data = np.load('MOM_INIT_DAT.npy').astype(complex)*0.393430307

In [98]:
atom_num = 11
mode_num = 27
t_num = 4000

In [99]:
with open("PYRIDINE-VIBRATIONS-1.mol") as f:
    omegas = np.array(f.readlines()[atom_num+3:atom_num+mode_num+3]).astype(float)*2*np.pi*100*299792458*2.418884326509e-17

In [100]:
from scipy.integrate import simps

ts = np.linspace(0.01,40,4000,endpoint=True)

DATA_FIELD_OMEGA = []
DATA_FIELD_INIT_OMEGA = []
DATA_MOMENT_OMEGA = []
POL_X, POL_Y = [], []
POL_X_INIT, POL_Y_INIT = [], []
POL_X_MOMENT, POL_Y_MOMENT = [], []
MINUS_MODE, PLUS_MODE = [], []
MINUS_MODE_INIT, PLUS_MODE_INIT = [], []
MINUS_MODE_MOMENT, PLUS_MODE_MOMENT = [], []

for p in range(2):
    for mp in range(2):
        for m in range(mode_num):
            induced_dipole = moment_data[p][mp][m] - init_moment_data[p][mp][m]
            # Energy of 1 Hartree (au of energy) is 27.211 eV, gamma is 0.1 eV
            exp_factor = np.exp((1j*omegas[m]-0.1/27.211386245988)*ts)
            id_times_exp = np.multiply(induced_dipole,exp_factor)
            integral_x = simps(id_times_exp[0],ts)
            integral_y = simps(id_times_exp[1],ts)
            integral_z = simps(id_times_exp[2],ts)
            integral = [integral_x,integral_y, integral_z]
            
            if mp == 0:
                MINUS_MODE_MOMENT.append(integral)
            else:
                PLUS_MODE_MOMENT.append(integral)
                
            exp_factor = np.exp(1j*omegas[m]*ts)
            ef_times_exp = np.multiply(field_data[p][mp][m],exp_factor)
            integral_x = simps(ef_times_exp[0],ts)
            integral_y = simps(ef_times_exp[1],ts)
            integral_z = simps(ef_times_exp[2],ts)
            integral = [integral_x,integral_y, integral_z]
            
            if mp == 0:
                MINUS_MODE.append(integral)
            else:
                PLUS_MODE.append(integral)
                
            ef_times_exp = np.multiply(init_field_data[p][mp][m],exp_factor)
            integral_x = simps(ef_times_exp[0],ts)
            integral_y = simps(ef_times_exp[1],ts)
            integral_z = simps(ef_times_exp[2],ts)
            integral = [integral_x,integral_y, integral_z]
            
            if mp == 0:
                MINUS_MODE_INIT.append(integral)
            else:
                PLUS_MODE_INIT.append(integral)
            
    if p == 0: 
        POL_X_MOMENT.append(MINUS_MODE_MOMENT)
        POL_X_MOMENT.append(PLUS_MODE_MOMENT)
        POL_X.append(MINUS_MODE)
        POL_X.append(PLUS_MODE)
        POL_X_INIT.append(MINUS_MODE_INIT)
        POL_X_INIT.append(PLUS_MODE_INIT)
        
    else:
        POL_Y_MOMENT.append(MINUS_MODE_MOMENT)
        POL_Y_MOMENT.append(PLUS_MODE_MOMENT)
        POL_Y.append(MINUS_MODE)
        POL_Y.append(PLUS_MODE)
        POL_Y_INIT.append(MINUS_MODE_INIT)
        POL_Y_INIT.append(PLUS_MODE_INIT)      
    
    MINUS_MODE, PLUS_MODE = [], []
    MINUS_MODE_INIT, PLUS_MODE_INIT = [], []
    MINUS_MODE_MOMENT, PLUS_MODE_MOMENT = [], []
            
DATA_FIELD_OMEGA.append(POL_X)
DATA_FIELD_OMEGA.append(POL_Y)
DATA_FIELD_INIT_OMEGA.append(POL_X_INIT)
DATA_FIELD_INIT_OMEGA.append(POL_Y_INIT)            
DATA_MOMENT_OMEGA.append(POL_X_MOMENT)
DATA_MOMENT_OMEGA.append(POL_X_MOMENT)         

In [101]:
DATA_FIELD_OMEGA = np.array(DATA_FIELD_OMEGA)
print(DATA_FIELD_OMEGA.shape)
DATA_MOMENT_OMEGA = np.array(DATA_MOMENT_OMEGA)
print(DATA_MOMENT_OMEGA.shape)
DATA_FIELD_INIT_OMEGA = np.array(DATA_FIELD_INIT_OMEGA)
print(DATA_FIELD_INIT_OMEGA.shape)

(2, 2, 27, 3)
(2, 2, 27, 3)
(2, 2, 27, 3)


In [102]:
np.save("E_FIELD_OMEGA_DAT",DATA_FIELD_OMEGA)
np.save("MOM_OMEGA_DAT",DATA_MOMENT_OMEGA)
np.save("E_FIELD_INIT_OMEGA_DAT",DATA_FIELD_INIT_OMEGA)